<img src="https://www.rp.edu.sg/images/default-source/default-album/rp-logo.png" width="200" alt="Republic Polytechnic"/>

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/koayst-rplesson/SDGAI_LLMforGenAIApp_Labs/blob/main/L09/L09.ipynb)

# Setup and Installation

You can run this Jupyter notebook either on your local machine or run it at Google Colab.

* For local machine, it is recommended to install Anaconda and create a new development environment called `c3669c`.
* Pip/Conda install the libraries stated below when necessary.
---

# <font color='red'>ATTENTION</font>

## Google Colab
- If you are running this code in Google Colab, **DO NOT** store the API Key in a text file and load the key later from Google Drive. This is insecure and will expose the key.
- **DO NOT** hard code the API Key directly in the Python code, even though it might seem convenient for quick development.
- You need to enter the API key at python code `getpass.getpass()` when ask.

## Local Environment/Laptop
- If you are running this code locally in your laptop, you can create a env.txt and store the API key there.
- Make sure env.txt is in the same directory of this Jupyter notebook.
- You need to install `python-dotenv` and run the Python code to load in the API key.

---
```
%pip install python-dotenv

from dotenv import load_dotenv

load_dotenv('env.tx')
openai_api_key = os.getenv('OPENAI_API_KEY')
```
---

## GitHub/GitLab
- **DO NOT** `commit` or `push` API Key to services like GitHub or GitLab.



# Lesson 09

- LangChain is a framework built around LLMs.
- Framework offered as a Python or Javascript (Typescript) package.
- Use it to build chatbots, Generative Question-Answer (GQA), summarization and much more.
- Core idea is to “chain” together different components to create more advanced use cases around LLMs.
- Provides developers with a comprehensive set of tools to seamlessly combine multiple prompts working with LLMs effortlessly.

In [ ]:
%%capture --no-stderr
%pip install --quiet -U langchain
%pip install --quiet -U langchain-openai

In [ ]:
# langchain        0.3.11
# langchain-core   0.3.24
# langchain-openai 0.2.12
# openai           1.57.2
# pydantic         2.10.3

In [1]:
import getpass
import os

# setup the OpenAI API Key

# get OpenAI API key ready and enter it when ask
os.environ["OPENAI_API_KEY"] = getpass.getpass()

 ········


### A Simple LLM Application 

In [2]:
# load langchain libraries
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage

In [3]:
# why not a gpt-3.5-turbo-instruct ?
# https://github.com/BerriAI/litellm/issues/749
# gpt-3.5-turbo-instruct is an openai text completions model and so gets routed to completions not chat completions.

# in summary you will notice the patterns:
# model name = gpt-3.5-turbo-instruct --> text completion --> OpenAI
# model name - gpt-3.5-turbo --> chat completion --> ChatOpenAI

chat_model = ChatOpenAI(
    # don't need this if the OpenAI API Key is stored in the environment variable
    #openai_api_key="sk-proj-xxxxxxxxx",

    #model_name='gpt-3.5-turbo'
    model_name='gpt-4o-mini'
)

In [4]:
# setup message prompt
text = "What date is Singapore National Day?"
messages = [HumanMessage(content=text)]

In [5]:
# note that Chat Model takes in message objects as input and generate message object as output

response = chat_model.invoke(messages)
print(response.content)

Singapore National Day is celebrated on August 9th each year. It commemorates Singapore's independence from Malaysia, which occurred on that date in 1965.


### A Simple LLM Application With Prompt Template

In [6]:
# load langchain libraries
from langchain_openai import ChatOpenAI
from langchain_core.prompts.chat import ChatPromptTemplate

In [7]:
# initialise ChatModel with API key
chat_model = ChatOpenAI(
    model_name='gpt-4o-mini', 
    temperature=0.3
)

In [8]:
# Prompt template takes in raw user input ({input_language} and {output_language}) and 
# return a prompt that is ready to pass into a language model

system_template = "You are a helpful assistant that translates {input_language} to {output_language}."

human_template = "{text}"
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", human_template),
])

In [9]:
# trsnslate English to French

messages = chat_prompt.format_messages(
    input_language="English", 
    output_language="French", 
    text="I love programming."
)

response = chat_model.invoke(messages).content
print(response)

J'aime la programmation.


In [10]:
# translate English to Chinese

messages = chat_prompt.format_messages(
    input_language="English", 
    output_language="Chinese", 
    text="I love programming."
)

response = chat_model.invoke(messages).content
print(response)

我爱编程。


# Observation

- From the two sample codes you just run, you have learned how to create your first simple LLM application. 
- You learned how to work with language model(s) and how to create a prompt template

## PromptTemplate And LangChain Expression Language (LCEL)

In [11]:
# load langchain libraries
from langchain_openai import ChatOpenAI
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_core.output_parsers.string import StrOutputParser

In [12]:
# gpt-4 does not have 'instruct' model
# The chat models like gpt-4 and gpt-4-turbo are already instruction-following models.
# They are designed to interpret and respond effectively to instructions provided in the conversatio

llm = ChatOpenAI(
    model_name='gpt-4o-mini',
    temperature=0.7,
)

output_parser = StrOutputParser()

In [13]:
# setup template 

human_template = "Write {lines} sentences about {topic}."
prompt = ChatPromptTemplate.from_template(human_template)

lines_topic_dict = {
    "lines" : "3", 
    "topic": "Sir Stamford Raffles"
}

In [14]:
# Without piping to StrOutputParser
# StrOutputParser = OutputParser that parses LLMResult into the top likely string

lcel_chain_01 = prompt | llm

lcel_chain_01.invoke(lines_topic_dict)

AIMessage(content="Sir Stamford Raffles was a British statesman and the founder of modern Singapore, establishing the island as a strategic trading post for the British East India Company in 1819. His vision extended beyond commerce; he was also a naturalist and played a significant role in promoting the study of Southeast Asian flora and fauna. Raffles' legacy is marked by his contributions to the region's development, his advocacy for education, and his efforts to preserve local culture and heritage.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 93, 'prompt_tokens': 17, 'total_tokens': 110, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_6fc10e10eb', 'finish_reason': 'stop', 'logprobs': None}, id='run-2abe430e-8af3-

In [15]:
# Pipe to StrOutputParesr

lcel_chain_02 = prompt | llm | output_parser

lcel_chain_02.invoke(lines_topic_dict)

"Sir Stamford Raffles was a British statesman and the founder of modern Singapore, establishing the island as a key trading post for the British East India Company in 1819. His vision extended beyond commerce; he was also an advocate for education, cultural preservation, and the welfare of local populations. Raffles' contributions to the region have left a lasting legacy, making him a pivotal figure in Southeast Asian history."